In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import os
import json
import pandas as pd
import torchvision.transforms as T

from train.multi_hyperparameter import MultiHyperparameter
from custom_models.unet_original import UNet, UNetSmall, UNetLarge
from evaluate.cross_evaluator import SemanticCrossEvaluator
from train.unet_trainer import UnetTrainer
from datasets.semantic_dataset import SemanticDataset
from preprocessing.data_augment import DataAugmenter

C:\Users\silva\anaconda3\envs\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
cwd = os.getcwd()
ds1_path = os.path.join(cwd, '../data/cell_type_1')
ds2_path = os.path.join(cwd, '../data/cell_type_2')

cv_param = {'interval_img_out': 1,
            'num_images': 3,
            'device': th.device("cuda" if th.cuda.is_available() else "cpu"),
            'datasets_path': [ds1_path, ds2_path],
            'results_path': os.path.join(cwd, '../results'),
            'folds': [0, 1, 2, 3],
            'epochs_cv': 1,
            'epochs_ct': 1,
            'num_random_params': 3}

param = {'id': 0,
         'padding_mode': 'reflect',
         'out_classes': 2,
         'criterion': nn.CrossEntropyLoss(),
         'optimizer': MultiHyperparameter({'type': [
             # 'sgd',
             #'adam',
              'rmsprop',
             # 'asgd'
         ],
             'lr_factor': [
                 # 10,
                 # 4,
                 # 2,
                 1,
                 # 0.5,
                 # 0.25,
                 # 0.1
             ],
             'weight_decay': [
                 0,
                 1e-3,
                 #1e-5
             ]
         }).get_full_grid_params(),
         'augment_transform': [{'rotate': False, 'mirror': False, 'translate': False, 'pad': 0},
                               #{'rotate': True, 'mirror': True, 'translate': False, 'pad': 0},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 16},
                               #{'rotate': True, 'mirror': True, 'translate': True, 'pad': 8}
                               ],
         'num_augments': 100,
         'binarizer_lr': 0.1,
         'batch_size': 2}

unet_hyps = MultiHyperparameter(param)
params = unet_hyps.get_full_grid_params(indexed=True)
print(len(params))
unet = UNet.__new__(UNet)

2


In [ ]:
cte = SemanticCrossEvaluator(unet, cv_param, 3)
print(params)
report = cte.cross_test_model(params, cv_param['epochs_ct'], cv_param['epochs_cv'])
#cte.train_validate(params[0], ds1_path, [1, 2], [3], 10)

[{'id': 0, 'padding_mode': 'reflect', 'out_classes': 2, 'criterion': CrossEntropyLoss(), 'optimizer': {'type': 'rmsprop', 'lr_factor': 1, 'weight_decay': 0}, 'augment_transform': {'rotate': False, 'mirror': False, 'translate': False, 'pad': 0}, 'num_augments': 100, 'binarizer_lr': 0.1, 'batch_size': 2}, {'id': 1, 'padding_mode': 'reflect', 'out_classes': 2, 'criterion': CrossEntropyLoss(), 'optimizer': {'type': 'rmsprop', 'lr_factor': 1, 'weight_decay': 0.001}, 'augment_transform': {'rotate': False, 'mirror': False, 'translate': False, 'pad': 0}, 'num_augments': 100, 'binarizer_lr': 0.1, 'batch_size': 2}]
Testing Fold: 0 / 3
Evaluating Param: 0 / 1
Evaluating Dataset: 0 / 1
Cross validate: 0 / 2


Training: 100%|██████████████████████████████| 1/1 [00:25<00:00, 25.08s/it]


Loss: 0.5746335393190384 Acc: 0.043727703392505646
Cross validate: 1 / 2


Training: 100%|██████████████████████████████| 1/1 [00:20<00:00, 20.31s/it]


Loss: 0.646323903799057 Acc: 0.07345665991306305
Cross validate: 2 / 2


...


Training: 100%|██████████████████████████████| 1/1 [00:21<00:00, 21.23s/it]

Loss: 0.5975966036319733 Acc: 0.5425939559936523
Cross validate: 2 / 2



Training: 100%|██████████████████████████████| 1/1 [00:20<00:00, 20.64s/it]


Loss: 0.5580284249782562 Acc: 0.5646120309829712
Evaluating Param: 1 / 2
Evaluating Dataset: 0 / 1
Cross validate: 0 / 2


Training:   0%|                                      | 0/1 [00:00<?, ?it/s]